# **Author: Aritra Ray**
Description: Using Pix2PixHD Model for Next Frame in Sequence Prediction, on my own video! 


Details of Pix2PixHD: https://github.com/NVIDIA/pix2pixHD

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Check to see what GPU we’re assigned

In [ ]:
!nvidia-smi -L

GPU 0: A100-SXM4-40GB (UUID: GPU-a1f469b5-b936-6e94-1420-91d5bab9318b)


## Install libraries and dependencies



In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/nfp-colab/pix2pixHD/"):
    %cd /content/drive/MyDrive/nfp-colab/pix2pixHD/
    # !git pull
    !pip install dominate
else:
    %cd /content/drive/MyDrive
    !mkdir nfp-colab
    %cd nfp-colab
    !git clone -b video https://github.com/dvschultz/pix2pixHD.git
    !pip install dominate
    %cd pix2pixHD/

/content/drive/MyDrive/nfp-colab/pix2pixHD
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Extract frames from video

Upload a video to either Colab or Google Drive.

* `-video` is the path to the video file
* `-name` should be a unique name (think of it like a project name)
* `-width` and `-height` **must** to be a multiple of 32
* `-p2pdir` leave as `.` unless you know it shouldn’t be that ;)


In [ ]:
!python extract_frames.py -video /content/aritra-ray.mp4 -name aritra-ray -p2pdir . -width 1280 -height 736

creating the dataset structure
ffmpeg -v 16 -i /content/aritra-ray.mp4 -q:v 2 -vf "scale=iw*736/ih:736, crop=1280:736" /content/drive/MyDrive/nfp-colab/pix2pixHD/datasets/aritra-ray/train_frames/frame-%06d.jpg -hide_banner
extracting the frames


## Train your model

### Initial training

Note: if you have a large dataset, this may timeout initially (`ValueError: __len__() should return >= 0`). Give it a minute or two and run it again.

*   `--name` should be a unique name (think of it like a project name). For your sanity I recommend using the same `-name` as above.
*   `--dataroot` should point to your dataset. This will usually be in `./datasets/[your project name]`

 



In [ ]:
!python train_video.py --name aritra-ray --dataroot ./datasets/aritra-ray/ --save_epoch_freq 5 #--continue_train

------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: False
data_type: 32
dataroot: ./datasets/aritra-ray/
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
fps: 24.0
gpu_ids: [0]
heat_seeking_lvl: 0
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 35
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: aritra-ray
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: False
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
pstart: 1
pstop: 1
resize_or_crop: scale_width
save_epoch_freq: 5
sa

### Continue Training
To pick up from a previous training session run the same command you ran to start with and append `--continue_train` to the end of the command.

In [ ]:
!python train_video.py --name aritra-ray --dataroot ./datasets/aritra-ray/ --save_epoch_freq 1 --continue_train 

#Generating Videos

To generate a video from your completed model, run the `generate_video.py` script. I recommend keeping the `--name` and `--dataroot` arguments the same.

There are a number of additional arguments you’ll need to include in this command:


*   `--fps` frame rate for your video
*   `--how_many` how many frames you want to create (this + fps = video length)
*   `--which_epoch` which epoch you want to generate videos from (note: if you choose `133` but there’s no epoch 133 model file, this will throw an error)
*   `--start_from` by default your video will start predicting images from the 60th frame of your training set. You can pass in the path to a different frame to have it start from that frame

I recommend playing with both the `--which_epoch` and `--start_from` arguments as you can get dramatically different results by changing in the inputs here.





In [ ]:
!python generate_video.py --name aritra-ray --dataroot ./datasets/aritra-ray/ --fps 24 --how_many 600 --which_epoch 50 --start_from 'video' 

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: ./datasets/aritra-ray/
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
fps: 24.0
gpu_ids: [0]
heat_seeking_lvl: 0
how_many: 600
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 35
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: aritra-ray
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_crop: True
no_flip: False
no_instance: False
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
png: False
pstart: 1
pstop: 1
resize_or_crop: scale_width
results_dir: ./results/
scheduled_sampling: False
serial_batches: False
ss_recursion_prob: 0.2
start_frames_before: 1
start_from: video
tf_l